# Creacion del dataframe para el modelo

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import re


In [9]:
dfgames = pd.read_csv(r'C:\Users\user\OneDrive\Escritorio\SOYHENRY\Curso Data Science\PI1\dfgames.csv')

In [10]:
dfgames.columns

Index(['publisher', 'genres', 'title', 'release_date', 'tags', 'specs',
       'price', 'early_access', 'id', 'developer', 'year'],
      dtype='object')

In [11]:
## El modelo se hace en base a las columnas tags(etiquetas), specs(especificaciones), y genres(genero).
todrop = ['publisher','release_date','price','early_access','year','developer']
dfgames.drop(columns = todrop, inplace= True)

In [12]:
dfgames

,genres,title,tags,specs,id
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,"['Strategy', 'Action', 'Indie', 'Casual', 'Sim...",['Single-player'],761140.0
1,"['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,"['Free to Play', 'Strategy', 'Indie', 'RPG', '...","['Single-player', 'Multi-player', 'Online Mult...",643980.0
2,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Real Pool 3D - Poolians,"['Free to Play', 'Simulation', 'Sports', 'Casu...","['Single-player', 'Multi-player', 'Online Mult...",670290.0
3,"['Action', 'Adventure', 'Casual']",弹炸人2222,"['Action', 'Adventure', 'Casual']",['Single-player'],767400.0
4,"['Action', 'Adventure', 'Simulation']",Battle Royale Trainer,"['Action', 'Adventure', 'Simulation', 'FPS', '...","['Single-player', 'Steam Achievements']",772540.0
...,...,...,...,...,...
29777,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,"['Action', 'Indie', 'Casual', 'Violent', 'Adve...","['Single-player', 'Steam Achievements', 'Steam...",745400.0
29778,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,"['Strategy', 'Indie', 'Casual', 'Simulation']","['Single-player', 'Steam Achievements']",773640.0
29779,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,"['Strategy', 'Indie', 'Casual']","['Single-player', 'Steam Achievements', 'Steam...",733530.0
29780,"['Indie', 'Racing', 'Simulation']",Russian Roads,"['Indie', 'Simulation', 'Racing']","['Single-player', 'Steam Achievements', 'Steam...",610660.0


In [21]:
# Concatenar las columnas de texto para la representación
dfgames['description'] = dfgames['genres'] + ' ' + dfgames['specs']+ ' ' + dfgames['tags'] + dfgames['title']

In [23]:
# Seleccionar las columnas 'description' y 'title'
dfmodelo = dfgames[['description', 'title']]


In [32]:
dfmodelo.to_csv('dfmodelo.csv')

In [29]:
def normalize_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', ' ', title).lower().strip()

def recommend_games(raw_title):
    # Normalizar el título ingresado
    title = normalize_title(raw_title)
    
    if 'normalized_title' not in dfmodelo.columns:
        dfmodelo.loc[:, 'normalized_title'] = dfmodelo['title'].apply(normalize_title)

    if title not in dfmodelo['normalized_title'].values:
        print(f"El juego '{raw_title}' no fue encontrado en la base de datos.")
        return
    
    # Usar TF-IDF para convertir el texto en vectores
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(dfgames['description'])

    # Crear el modelo de vecinos más cercanos
    knn = NearestNeighbors(n_neighbors=6, metric='cosine', algorithm='brute')  
    knn.fit(tfidf_matrix)

    game_index = dfmodelo[dfmodelo['normalized_title'] == title].index[0]
    
    distances, indices = knn.kneighbors(tfidf_matrix[game_index], n_neighbors=6)  
    
    recommended_games = dfmodelo.iloc[indices.flatten()[1:]]  
    
    print(f"Las recomendaciones para '{raw_title}' son:")
    for i, (index, row) in enumerate(recommended_games.iterrows(), start=1):
        print(f"{i}: {row['title']}")
    return

In [30]:
#Ejemplos de recomendacion
recommend_games('Ironbound')

Las recomendaciones para 'Ironbound' son:
1: Infinity Wars: Animated Trading Card Game
2: Chronicle: RuneScape Legends
3: Faeria
4: Kingdoms CCG
5: HEX: Shards of Fate


In [31]:
recommend_games('The first thrust of God')

Las recomendaciones para 'The first thrust of God' son:
1: The first thrust of God - All Aircrafts
2: Salt Thrust
3: The Sandbox
4: Mitosis
5: God Simulator
